## Mapping Colonial Medical Expertise of the Old Regime ##
### Sean Takats, George Mason University ###

In *la zone torride*, where French colonial efforts were effectively confined after the Seven Years War, profitable commerce and brutal mortality rates combined to produce a steady supply of desperate but paying patients. Just as the colonies of the Old Regime provided the chance for planters and traders to grow rich (faire fortune), they also promised to reward doctors who could keep those individuals healthy in the face of endemic disease. Chasing fame and wealth, hundreds of medical practitioners fanned out across the globe from France to support a growing colonial and military infrastructure. At the same time, they built up a new body of medical knowledge that might prove useful in the metropole, where neo-Hippocratic conceptions of disease and hygiene were pushed to their limits by colonial findings.

Despite perceived importance of this endeavor, with a range of new royal institutions established to support it, few historical studies have treated colonial medical practitioners apart from their published medical treatises, and medical histories of the Old Regime largely neglect colonial medicine. This neglect reflects at least in part a methodological bias, which this paper addresses. While many of doctors recounted their colonial experiences in published articles and books, the vast majority did not, and records of their experiences remain largely ignored in the archives. Only by considering it in its totality can we regain a sense of the centrality of this exotic experience to contemporary medical practice.

This paper seeks to visualize what that expertise might have looked like over the course of the eighteenth century. By computationally processing digital records of the Archives nationales d’Outre-Mer, I generate preliminary maps that suggest the development of increasingly dense and complex patterns of tropical medical expertise after 1750. I draw on approximately 20,000 records of French colonial agents and hundreds of medical practitioners included among them to explore the shifting centers of gravity of the colonial medical enterprise over the course of the eighteenth century. By seeking a concordance of medical expertise with the location in time and space of the experts themselves, we will have a clearer sense of how colonial medicine happened on the ground.

## Data Processing ##
I downloaded and processed all 19,338 entries in the finding aid for [Série E of the Archives nationales d’Outre-Mer](http://anom.archivesnationales.culture.gouv.fr/ark:/61561/up424ojc). This finding aid indexes 63 linear meters of records, stored in 441 cartons. Each record in the finding aid looks something like this: 

*Artur, Jacques François, médecin et doyen du Conseil supérieur à Cayenne 1735/1781*

We can immediately see that there’s a significant amount of information in this record that could be useful to us: name, occupation, location, and time. Each record in the finding aid contains at least a name, and most contain a full complement of data, often with multiple locations (more on this later). 

A finding aid like this one, however, is typically used to conduct a search. And search as a research strategy is, of course, problematic. The finding aid itself is not granted any particular significance or use in its own right. For example, if I wanted to find out something about doctors, I might search for “médecin” and find some records, including Artur’s. Or I might be interested in Guyane and search “Cayenne”, again finding his record. But both of those searches presuppose a great deal. I’ve already decided to look for “Cayenne” but how do I even know if Guyana is important at all? Things get even more complicated if I want to think about time. How can I find records of individuals that intersect with a certain period? To make such queries possible, we can instead consider the finding aid in the aggregate, to identify times and places that may encourage further investigation. In other words, why not attempt a distant reading of the records, via computational analysis of their metadata?

Once I had loaded all the records into a database, I then extracted the metadata from each record: personal name, place name(s), and military corps name (persname, geogname, and corpname, in ANOM’s schema), along with any years provided. With the Series E records now searchable, sortable, and (most important) containing structured data, I could then begin to view the dataset in the aggregate. At this stage, however, we are still working with fundamentally qualitative data (other than the years). 

In [1]:
import re, sys
import pandas as pd, numpy as np
import pickle
from bs4 import BeautifulSoup

In [2]:
# I'm currently storing my downloaded data from ANOM using jupyter persistence
# we'll load personneldf from here
%store -r

## Filter "medical" personnel ##
My next step was to pass the database of colonial agents through a filter to identify medical personnel, flagging any record that included “santé”, “médic”, “médecin”, or “chirurg”, a process which yields 526 records. With these records in hand and a geocoded database of locations, we can now plot these colonial personnel in time and space.

In [3]:
medicaldf = personneldf[personneldf.fullRecord.str.contains('santé|médic|médecin|chirurg')]

In [4]:
medicaldf

,archref,name,placesdisplay,placesnormalized,corp,startdate,enddate,daterange,alldates,fullLink,fullRecord
85,"<a class=""archref"" href=""/ark:/61561/up424g0z0...","Akers, Arétas",[Saint-Vincent],"[Saint-Vincent, Île (Antilles)]",,1783,1783,1783-1783,1783,/ark:/61561/up424g0z0c,"Akers, Arétas, habitant de l'île Saint-Vincent..."
89,"<a class=""archref"" href=""/ark:/61561/up424rlkp...",Alavoine,[Trois-Rivières],[Trois-Rivières (Canada)],Hôpital général (Trois-Rivières ; Canada),1757,1761,1757-1761,"1757, 1761",/ark:/61561/up424rlkps,"Alavoine, chirurgien-major à l'hôpital des Tro..."
102,"<a class=""archref"" href=""/ark:/61561/up424xrry...","Albert, Nicolas",NaN,NaN,,1786,1786,1786-1786,1786,/ark:/61561/up424xrryu,"Albert, Nicolas, botaniste ayant le privilège ..."
118,"<a class=""archref"" href=""/ark:/61561/up424uoqr...",Aleaume,[Sainte-Lucie],"[Sainte-Lucie, Île (Antilles)]",,1763,1763,1763-1763,1763,/ark:/61561/up424uoqrg,"Aleaume, docteur en médecine à Sainte-Lucie 1763"
190,"<a class=""archref"" href=""/ark:/61561/up424rmkq...","Amelin de Lacroix, Charles",[Saint-Domingue],"[Saint-Domingue, Colonie française (Saint-Domi...",,1770,1770,1770-1770,1770,/ark:/61561/up424rmkqu,"Amelin de Lacroix, Charles, chirurgien, passé ..."
245,"<a class=""archref"" href=""/ark:/61561/up424pkoj...","Angerville, Côme d'",[Cap-Français],"[Cap-Français (Saint-Domingue, Île de)]",,1778,1780,1778-1780,"1778, 1780",/ark:/61561/up424pkojn,"Angerville, Côme d', chirurgien du Roi, au Cap..."
264,"<a class=""archref"" href=""/ark:/61561/up424niog...","Annibal, François","[Indes, Chandernagor]","[Inde française, Chandernagor (Inde)]",None,1777,1778,1777-1778,"1777, 1778",/ark:/61561/up424niogg,"Annibal, François, novice du vaisseau le Borde..."
267,"<a class=""archref"" href=""/ark:/61561/up424toup...","Ansermet, Jean Louis",[île de France],"[France, Île de (Mascareignes, Îles)]",,1774,1789,1774-1789,"1774, 1789",/ark:/61561/up424touph,"Ansermet, Jean Louis, chirurgien à l'île de Fr..."
301,"<a class=""archref"" href=""/ark:/61561/up424dzx3...",Archambault,[Madagascar],"[Madagascar, Île de]",,1789,1789,1789-1789,1789,/ark:/61561/up424dzx3w,"Archambault, proposé pour chirurgien-major à M..."
372,"<a class=""archref"" href=""/ark:/61561/up424rnss...","Arnoux, André",[Québec],[Québec (Canada)],,1746,1746,1746-1746,1746,/ark:/61561/up424rnssf,"Arnoux, André, chirurgien-major du Roi à Québe..."


## Geocode Place Names ##

My next step was to geocode the place names embedded in each record. I extracted all the place names from my database of colonial personnel, rendering a list of 1401 unique location names. Fortunately ANOM offers a geographic lookup service, [Assistant de recherche géographique](http://anom.archivesnationales.culture.gouv.fr/geo.php), which provides GPS coordinates for the vast majority of these locations. I wrote a small piece of code to look up each location in the Assistant de recherche and then add the returned coordinates to my list of unique locations. I entered the small number of unreturned locations by hand using Wikipedia's geocodes.

For the purposes of this analysis, we'll only use those locations which are associated with at least one medical practitioner, i.e. ones that match a record included in our medicaldf data frame.

In [5]:
medicalgeocodes = pd.DataFrame.from_csv('/Users/stakats/Downloads/anommedicalgeocoded-edited.csv')

In [6]:
medicalgeocodes

,normalized,lat,lng
0,"Bourbon, Île (Mascareignes, Îles)",-21.150000,55.500000
1,Surate (Inde),21.195000,72.819400
2,Landau (Allemagne),51.344100,9.084000
3,Chandernagor (Inde),22.855900,88.380000
4,La Balise (Louisiane ; États-Unis),29.126400,-89.088100
5,"Artois, Région géographique (France)",50.500000,2.500000
6,"Fort-Royal (Martinique, Île de la)",14.610800,-61.071800
7,Guérande (Loire-Atlantique ; France),47.333300,-2.433300
8,Boën (Loire ; France),45.744500,4.005000
9,"Saint-Pierre (Martinique, Île de la)",14.742500,-61.175000


## Get ready to visualize ##
Now we're finally ready to see something. We'll begin by defining some very basic mapping functions.

In [40]:
from IPython.display import HTML
import folium
 
def inline_map(map):
    """
    Embeds the HTML source of the map directly into the IPython notebook.
    
    This method will not work if the map depends on any files (json data). Also this uses
    the HTML5 srcdoc attribute, which may not be supported in all browsers.
    """
    map._build_map()
    return HTML('<iframe srcdoc="{srcdoc}" style="width: 100%; height: 510px; border: none"></iframe>'.format(srcdoc=map.HTML.replace('"', '&quot;')))
 
def embed_map(map, path):
    """
    Embeds a linked iframe to the map into the IPython notebook.
    
    Note: this method will not capture the source of the map into the notebook.
    This method should work for all maps (as long as they use relative urls).
    """
    map.create_map(path=path)
    return HTML('<iframe src="files/{path}" style="width: 100%; height: 510px; border: none"></iframe>'.format(path=path))

def embed_map_test(map, path):
    HTML('<iframe src="'+path+'" style="width: 100%; height: 510px; border: none"></iframe>')

First let's map the all the unique locations that could appear on the map.

In [42]:
# We could use any number of pretty map backgrounds, but this one is relatively anodyne
tileset = 'http://{s}.tile.openstreetmap.se/hydda/base/{z}/{x}/{y}.png'
attribution = 'Tiles courtesy of <a href="http://openstreetmap.se/">OpenStreetMap Sweden</a> &mdash; Map data &copy; <a href="http://www.openstreetmap.org/copyright">OpenStreetMap</a>'

fmap=folium.Map(location=[0, 0], zoom_start=2, tiles=tileset, attr=attribution)
def plotmarker(row):
    fmap.simple_marker( [row['latitude'], row['longitude']] )
for row in medicalgeocodes.iterrows():
    latlon = [ row[1]['lat'], row[1]['lng'] ]
    fmap.simple_marker(latlon, popup = row[1]['normalized'])
fmap.create_map('test.html')
HTML('<iframe src="test.html" style="width: 100%; height: 510px; border: none"></iframe>')

Now let's map the medical personnel themselves.

In [9]:
fmap=folium.Map(location=[0, 0], zoom_start=2, tiles=tileset, attr=attribution)
def plotmarker(row):
    fmap.simple_marker( [row['latitude'], row['longitude']] )
for row in medicaldf.iterrows():
    places = row[1]['placesnormalized']
    if isinstance(places, list):
        linepoints = []
        for place in places:
            result = medicalgeocodes.loc[medicalgeocodes['normalized'] == place]
            if not (result.empty):
                latlng = [ result['lat'].item(), result['lng'].item() ]
                linepoints.append(latlng)
                fmap.simple_marker(latlng, popup='<a href="http://anom.archivesnationales.culture.gouv.fr/'+
                                   row[1]['fullLink']+'">'+
                                   row[1]['fullRecord']+'</a>')
embed_map(fmap, 'all-med-people.html')

With so many points overlapping, this map is difficult to read. We can cluster the markers in order to gain a better sense of how they're distributed.

In [10]:
fmap=folium.Map(location=[0, 0], zoom_start=2, tiles=tileset, attr=attribution)
def plotmarker(row):
    fmap.simple_marker( [row['latitude'], row['longitude']] )
for row in medicaldf.iterrows():
    places = row[1]['placesnormalized']
    if isinstance(places, list):
        linepoints = []
        for place in places:
            result = medicalgeocodes.loc[medicalgeocodes['normalized'] == place]
            if not (result.empty):
                latlng = [ result['lat'].item(), result['lng'].item() ]
                linepoints.append(latlng)
                fmap.simple_marker(latlng, popup='<a href="http://anom.archivesnationales.culture.gouv.fr/'+
                                   row[1]['fullLink']+'">'+
                                   row[1]['fullRecord']+'</a>', clustered_marker=True)
embed_map(fmap, 'all-med-people-clustered.html')

What do we see? First, by geocoding locations, we immediately get a much clearer view of where colonial personnel physically were: while many records include obvious keywords, like Saint-Domingue, many others do not; instead they might only include the name of a river, at whose mouth a doctor was posted. Once we translate these records into latitude and longitude, we gain a much better sense of the spatial arrangement of medical personnel.

These points cluster tightly around the key nodes of French colonial activity: the Caribbean (56%), North America (10%), Madagascar and the Mascarenes (9%), West Africa (7%), and India (5%). The remaining points, typically identifying the individual’s place of origin, fall in and around France (13%). The overwhelming importance of the Caribbean will come as little surprise, given the economic importance of sugar production and the intensive efforts to maintain healthy military and civilian and slave populations. Once we begin to examine them in slices (e.g. by decade), however, we find evidence of a significant shift in the geography of colonial medical practitioners, and in their mobility.

We notice immediately that the total number of points significantly exceeds our initial total set of 526 "medical" personnnel. Plotting all 526 records 636 points on the map because some individuals have more than one location associated with them in the finding aid entry. Most notably there are 82 records "located" in France, usually because these records include the individual's place of origin, as well as a colonial location. We can identify these records pretty quickly just by drawing lines between the points of any records that contain more than one location.

In [11]:
fmap=folium.Map(location=[0, 0], zoom_start=2, tiles=tileset, attr=attribution)
def plotmarker(row):
    fmap.simple_marker( [row['latitude'], row['longitude']] )
for row in medicaldf.iterrows():
    places = row[1]['placesnormalized']
    if isinstance(places, list):
        linepoints = []
        for place in places:
            result = medicalgeocodes.loc[medicalgeocodes['normalized'] == place]
            if not (result.empty):
                latlng = [ result['lat'].item(), result['lng'].item() ]
                linepoints.append(latlng)
        if len(linepoints) > 1:
            fmap.line(locations=linepoints, line_weight=2, line_opacity=0.4)
embed_map(fmap, 'all-networks.html')



Once we connect the dots, however, it becomes evident that many records link locations *among* the colonies, rather than simply between a colony and France. We see somethng that looks like a belt tying together the Caribbean, Madagascar, and India. This phenomenon is worth investigating further, so let's be sure to include line generation in our generic mapping function.

It is important to note that these lines are not geographic trajectories, strictly speaking. They don’t tell us where or when someone started moving, or in what direction they traveled. They don’t tell us how long someone stayed in a given location or even what they did there. In fact, they don’t even tell us whether someone ever moved at all. Jacques-Antoine Ribet’s dossier, for example, lists Martinique and Île de Bourbon merely because Ribet was actively lobbying for a position in one of those two locations, not because he had actually served in them.

In [12]:
# Let's now make a copy of our medical data frame so that we can do some destructive operations on it.
medicaldf_sample_test = medicaldf

In [13]:
# remove  blank dates
medicaldf_sample_test = medicaldf_sample_test.loc[medicaldf_sample_test['enddate'] != '']

In [14]:
# convert dates to integers so we can compare them programmatically
# we'll get a warning here but ignore it
medicaldf_sample_test.startdate = medicaldf_sample_test.startdate.astype(int)
medicaldf_sample_test.enddate = medicaldf_sample_test.enddate.astype(int)


/usr/local/lib/python3.5/site-packages/pandas/core/generic.py:2177: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


We've done some basic plotting of individuals on the map. Now let's see how things move over time. First we'll break our data sample up into slices by decade, with slightly longer periods at either end where things taper off.

In [15]:
medicaldf_sample_1690_1720 = medicaldf_sample_test[(1690 <= medicaldf_sample_test["enddate"]) & (medicaldf_sample_test["startdate"] <= 1720)]
medicaldf_sample_1720_1730 = medicaldf_sample_test[(1720 <= medicaldf_sample_test["enddate"]) & (medicaldf_sample_test["startdate"] <= 1730)]
medicaldf_sample_1730_1740 = medicaldf_sample_test[(1730 <= medicaldf_sample_test["enddate"]) & (medicaldf_sample_test["startdate"] <= 1740)]
medicaldf_sample_1740_1750 = medicaldf_sample_test[(1740 <= medicaldf_sample_test["enddate"]) & (medicaldf_sample_test["startdate"] <= 1750)]
medicaldf_sample_1750_1760 = medicaldf_sample_test[(1750 <= medicaldf_sample_test["enddate"]) & (medicaldf_sample_test["startdate"] <= 1760)]
medicaldf_sample_1760_1770 = medicaldf_sample_test[(1760 <= medicaldf_sample_test["enddate"]) & (medicaldf_sample_test["startdate"] <= 1770)]
medicaldf_sample_1770_1780 = medicaldf_sample_test[(1770 <= medicaldf_sample_test["enddate"]) & (medicaldf_sample_test["startdate"] <= 1780)]
medicaldf_sample_1780_1800 = medicaldf_sample_test[(1780 <= medicaldf_sample_test["enddate"]) & (medicaldf_sample_test["startdate"] <= 1800)]

Now let's build a generic mapping function that will let us map one of these samples, draw lines, and/or cluster the data points.

In [16]:
def map_anom(df, centered, zoom, lines, clustered):
    """
    Takes ANOM records, geocodes them according to normalized data
    variables are df, centered coordinates, zoom level, connecting lines, clustered points)
    """
    fmap=folium.Map(location=centered, zoom_start=zoom, tiles=tileset, attr=attribution)
    def plotmarker(row):
        fmap.simple_marker( [row['latitude'], row['longitude']] )
    for row in df.iterrows():
        places = row[1]['placesnormalized']
        if isinstance(places, list):
            linepoints = []
            for place in places:
                result = medicalgeocodes.loc[medicalgeocodes['normalized'] == place]
                if not (result.empty):
                    latlng = [ result['lat'].item(), result['lng'].item() ]
                    linepoints.append(latlng)
                    fmap.simple_marker(latlng, popup='<a href="http://anom.archivesnationales.culture.gouv.fr/'+
                                       row[1]['fullLink']+'">'+
                                       row[1]['fullRecord']+'</a>', clustered_marker=clustered)
            if lines & (len(linepoints) > 1):
                fmap.line(locations=linepoints, line_weight=2, line_opacity=0.4)
            
    return fmap

## French Colonial Medical Practitioners, 1690-1720 ##

In [17]:
fmap = map_anom(medicaldf_sample_1690_1720, [0, 0], 2, True, True)
embed_map(fmap, '1690-1720.html')

## French Colonial Medical Practitioners, 1720-1730 ##

In [18]:
fmap = map_anom(medicaldf_sample_1720_1730, [0, 0], 2, True, True)
embed_map(fmap, '1720-1730.html')

## French Colonial Medical Practitioners, 1730-1740 ##

In [19]:
fmap = map_anom(medicaldf_sample_1730_1740, [0, 0], 2, True, True)
embed_map(fmap, '1730-1740.html')

## French Colonial Medical Practitioners, 1740-1750 ##

In [20]:
fmap = map_anom(medicaldf_sample_1740_1750, [0, 0], 2, True, True)
embed_map(fmap, '1740-1750.html')

## French Colonial Medical Practitioners, 1750-1760 ##

In [21]:
fmap = map_anom(medicaldf_sample_1750_1760, [0, 0], 2, True, True)
embed_map(fmap, '1750-1760.html')

## French Colonial Medical Practitioners, 1760-1770 ##

In [22]:
fmap = map_anom(medicaldf_sample_1760_1770, [0, 0], 2, True, True)
embed_map(fmap, '1760-1770.html')

## French Colonial Medical Practitioners, 1770-1780 ##

In [23]:
fmap = map_anom(medicaldf_sample_1770_1780, [0, 0], 2, True, True)
embed_map(fmap, '1770-1780.html')

## French Colonial Medical Practitioners, 1780-1800 ##

In [24]:
fmap = map_anom(medicaldf_sample_1780_1800, [0, 0], 2, True, True)
embed_map(fmap, '1780-1800.html')

### Discussion of mapping over time ###
Once we begin to examine them in slices (e.g. by decade), however, we find evidence of a significant shift in the geography of colonial medical practitioners, and perhaps in their mobility.

In absolute terms, the number of medical practitioners increases significantly decade by decade, with an important inflection point around 1770, with records of medical practitioners doubling nearly every decade through the end of the Old Regime.

As the numbers grow the allocation by region shifts: by 1770 15% are in West Africa and the Indian Ocean, up from essentially zero, and by 1780 over a quarter are there, a presence maintained through the end of the Old Regime.

## Investigating "French" Locations ##
What can we learn by looking at the data points located in France? Like the multiple data points that produced lines, these "French" pieces of data initially seemed superfluous or even annoying. But let's take a closer see to how "France" appears when we look more closely. There aren't any "French" locations listed before 1720, so we'll begin with that slice of the data.

### Origins, 1720-1730 ###

In [25]:
fmap = map_anom(medicaldf_sample_1720_1730, [45.7831, 3.0824], 6, False, False)
embed_map(fmap, 'origins-1720-1730.html')

### Origins, 1730-1740 ###

In [26]:
fmap = map_anom(medicaldf_sample_1730_1740, [45.7831, 3.0824], 6, False, False)
embed_map(fmap, 'origins-1730-1740.html')

### Origins, 1740-1750 ###

In [27]:
fmap = map_anom(medicaldf_sample_1740_1750, [45.7831, 3.0824], 6, False, False)
embed_map(fmap, 'origins-1740-1750.html')

### Origins, 1750-1760 ###

In [28]:
fmap = map_anom(medicaldf_sample_1750_1760, [45.7831, 3.0824], 6, False, False)
embed_map(fmap, 'origins-1750-1760.html')

### Origins, 1760-1770 ###

In [29]:
fmap = map_anom(medicaldf_sample_1760_1770, [45.7831, 3.0824], 6, False, False)
embed_map(fmap, 'origins-1760-1770.html')

### Origins, 1770-1780 ###

In [30]:
fmap = map_anom(medicaldf_sample_1770_1780, [45.7831, 3.0824], 6, False, False)
embed_map(fmap, 'origins-1770-1780.html')

### Origins, 1780-1800 ###

In [31]:
fmap = map_anom(medicaldf_sample_1780_1800, [45.7831, 3.0824], 6, False, False)
embed_map(fmap, 'origins-1780-1800.html')

## Origins Discussion ##
Before 1750, most medical practitioners' origins are clustered in port cities or near centers of medical learning like Paris and Montpellier. From 1760 onward, we find increasing numbers of medical practitioners originating elsewhere in central and eastern France. This diversification of geographic origins demands further investigation.
